In [1]:
import sys
sys.path.append("../src")
import pandas as pd
import numpy as np
from tqdm import tqdm
import pickle
import lightgbm
import pickle
from dataLoad import testLoad
from metric import ndcg_calculator, hit_at_k

path= "../dataset/"
test, final_sumbission, test_user_label = testLoad(path)

In [2]:
test = test.sort_values(by ="timestamp", ascending = True)\
        .reset_index(drop=True).drop(columns=["timestamp","click_count_normalized","user_click_count_normalized"])

In [3]:
with open("../output/first_batch2000lgbm_model.pkl", 'rb') as f:
    first_batch2000lgbm_model = pickle.load(f)

--- LgbmRanker + ALS + MP + Word2Vec Ensemble      
      
feature :      
*    datset - basic features from featureExtract      
*    Mp - general most popular feature      
*    ALS - user / item factor      
<!-- *    Word2Vec - item segmentation       -->
      
candidate :      
*    Mp  - general/recent weeks Most popular      
<!-- *    Word2Vec - similar items       --> 

In [4]:
# # load features
basic_7_feature = pd.read_parquet("../output/basic_7_feature.parquet")
general_MP_feature = pd.read_parquet("../output/general_MP_feature.parquet")
user_factor_als = pd.read_parquet("../output/user_factor_als.parquet")
item_factor_als = pd.read_parquet("../output/item_factor_als.parquet")

# # load candidates
recent30days_MP_candidate = pd.read_parquet("../output/recent30days_MP_candidate.parquet")
popular_items_cand = pd.read_parquet("../output/popular_items_cand.parquet")

In [5]:
candidate_1 = recent30days_MP_candidate
candidate_2 = popular_items_cand
candidate = [candidate_1, candidate_2]
cand = pd.concat(candidate)
cand.drop_duplicates(subset=['user','item_id'],inplace=True)

print('cand 데이터 수:',len(cand))
print('cand nunique: ', cand.item_id.nunique())

cand 데이터 수: 119636648
cand nunique:  177


In [6]:
# add feature 
candidate = pd.merge(cand, general_MP_feature, how='left', on='item_id')
candidate = pd.merge(cand, basic_7_feature, how='left', on='item_id')
candidate_add_features = candidate.copy()

MemoryError: Unable to allocate 1.76 TiB for an array with shape (241847500280,) and data type int64

In [ ]:
'''
OOM issue
'''

In [ ]:
# als vector merge
candidate_add_features = pd.merge(candidate_add_features, item_factor_als, how="left", on="item_id")
candidate_add_features = pd.merge(candidate_add_features, user_factor_als, how="left", on="user")

# print('cand 피처 수:',len(candidate_add_features))
# OOM issue로 학습에서는 앙상블 불가

In [ ]:
candidate_add_features = candidate_add_features.fillna(0).astype(int)  # astype int, fillna

In [ ]:
candidate_add_features

In [ ]:
def make_label(train_valid):
    label_df = train_valid[["user","item_id"]]
    label_df.drop_duplicates(subset=["user","item_id"],inplace=True)
    label_df["click_num"] = 1
    return label_df

def label_create(train, label_df):
    train = pd.merge(train, label_df, how="left", on=["user","item_id"])
    train["click_num"] = train["click_num"].fillna(0)
    return train

In [ ]:
label_df = make_label(test_user_label)
candidate_add_features = label_create(candidate_add_features, label_df)

In [ ]:
# print(candidate_add_features[candidate_add_features["click_num"]==1].count()) # 46771
# print(candidate_add_features[candidate_add_features["click_num"]==0].count()) # 102260423

In [ ]:
def lgbm_preprocess(train):
    X_train = train.drop(columns=["click_num"])
    y_train = train["click_num"]
#     group_dict = train.groupby("user")["user"].count().to_dict()
    item_idx = X_train["item_id"].copy()
    user_idx = X_train["user"].copy()
    del X_train["item_id"], X_train["user"]
    return X_train, y_train, item_idx, user_idx

def make_groups(X_train):
    group_dict = []
    batchsize = len(X_train)
    while True:
        if batchsize >= 2000:
            group_dict.append(2000)
            batchsize = batchsize - 2000
        else:
            group_dict.append(batchsize)
            break
            
def train_LgbmRanker_batch(X_train, y_train, model_params, batch_size):
    model = first_batch2000lgbm_model
    num_batches = X_train.shape[0] // batch_size
    for i in tqdm(range(0, X_train.shape[0], batch_size), total=num_batches):
        X_batch = X_train[i:i + batch_size]
        y_batch = y_train[i:i + batch_size]
        group_dict = []
        batchsize = len(X_batch)
        while batchsize !=0 :
            if batchsize >= 1000:
                group_dict.append(1000)
                batchsize = batchsize - 1000
            else:
                group_dict.append(batchsize)
                batchsize -= batchsize 
        model.fit(
        X=X_batch,
        y=y_batch,
        group=group_dict,
        ) 
    feature_importances_df = pd.DataFrame(dict(zip(X_train.columns, model.feature_importances_)), index=["feature_importances"]).T
    return model, feature_importances_df

    
def valid_evaluation(X_train, train_valid, sample_sumbission, model, feature_importances_df, item_idx, user_idx): 
    print(feature_importances_df)
    pred = model.predict(X_train)
    X_train["pred"] = pred
    X_train["item_id"] = item_idx
    X_train["user"] = user_idx
    
    print("performance")
    # each user pred 10 items
    lgbm_sub_df = X_train.sort_values(by="pred", ascending=False).groupby("user").head(10)
    lgbm_user_items_dict = lgbm_sub_df.groupby("user")["item_id"].unique().to_dict()
    sample_sumbission["predicted_list"] = sample_sumbission["user"].apply(lambda x: lgbm_user_items_dict.get(x, []))

    # print("lgbm ndcg:", ndcg_calculator(sample_sumbission, train_valid, 10))
        
    return X_train, sample_sumbission

In [ ]:
import gc
gc.collect()

In [ ]:
model_params={"num_leaves":150 ,"learning_rate":0.005,"n_estimators":35}

In [ ]:
del candidate, test, recent30days_MP_candidate, popular_items_cand, general_MP_feature

In [ ]:
X_train, y_train, item_idx, user_idx = lgbm_preprocess(candidate_add_features)
model, feature_importances_df = train_LgbmRanker_batch(X_train, y_train, model_params, 10000)

X_train, final_sumbission = \
     valid_evaluation(X_train, test_user_label, final_sumbission, model, feature_importances_df, item_idx, user_idx)

In [ ]:
%%time
mf_ndcg = ndcg_calculator(train_valid, mf_preds, 10)

print("performance")
print(f"nDCG@10(mf_ALS): {mf_ndcg:.4f}")

In [ ]:
%%time
mf_hit = hit_at_k(train_valid, sample_sumbission, 10)

print("performance")
print(f"hit@10(mf_ALS): {mf_hit:.4f}")

In [ ]:
%%time
mf_hit = hit_at_k(train_valid, sample_sumbission, 1)

print("performance")
print(f"hit@1(mf_ALS): {mf_hit:.4f}")

In [ ]:
X_train.to_parquet("../output/batch1000_lgbm_X_train.parquet", index=False)
pickle.dump(model, open("../output/batch1000_lgbm_model.pkl", "wb"))